In [1]:
import time
import numpy as np
import tensorflow as tf

In [2]:
!mkdir data
!wget -q -O data/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
!unzip -o data/ptb.zip -d data
!cp data/ptb/reader.py .

import reader

Archive:  data/ptb.zip
   creating: data/ptb/
  inflating: data/ptb/reader.py      
   creating: data/__MACOSX/
   creating: data/__MACOSX/ptb/
  inflating: data/__MACOSX/ptb/._reader.py  
  inflating: data/__MACOSX/._ptb     


In [3]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C data/

--2019-12-27 10:44:39--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  4.04MB/s    in 9.6s    

2019-12-27 10:44:49 (3.45 MB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [4]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 60
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size = 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

In [5]:
session = tf.InteractiveSession()

In [6]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [7]:
len(train_data)

929589

In [8]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line            
                

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


Lets just read one mini-batch now and feed our network:

In [9]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
x = first_touple[0]
y = first_touple[1]

In [10]:
x.shape

(60, 20)

Lets look at 3 sentences of our input x:

In [11]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [ 901,   33, 3361,    8, 1279,  437,  597,    6,  261, 4276, 1089,
           8, 2836,    2,  269,    4, 5526,  241,   13, 2420],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605]],
      dtype=int32)

we define 2 place holders to feed them with mini-batchs, that is x and y:

In [12]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
_targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

Lets define a dictionary, and use it later to feed the placeholders with our first mini-batch:

In [13]:
feed_dict = {_input_data:x, _targets:y}

For example, we can use it to feed <code>\_input\_data</code>:

In [14]:
session.run(_input_data, feed_dict)

array([[9970, 9971, 9972, ..., 9993, 9994, 9995],
       [ 901,   33, 3361, ...,  241,   13, 2420],
       [2654,    6,  334, ...,  514,    8,  605],
       ...,
       [7831,   36, 1678, ...,    4, 4558,  157],
       [  59, 2070, 2433, ...,  400,    1, 1173],
       [2097,    3,    2, ..., 2043,   23,    1]], dtype=int32)

In this step, we create the stacked LSTM, which is a 2 layer LSTM network:

In [15]:
lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l1, forget_bias=0.0)
lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l2, forget_bias=0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

Also, we initialize the states of the nework:

<h4>_initial_state</h4>

For each LCTM, there are 2 state matrices, c\_state and m\_state.  c_state and m_state represent "Memory State" and "Cell State". Each hidden layer, has a vector of size 30, which keeps the states. so, for 200 hidden units in each LSTM, we have a matrix of size [30x200]

In [16]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(60, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(60, 256) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(60, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(60, 128) dtype=float32>))

Lets look at the states, though they are all zero for now:

In [17]:
session.run(_initial_state, feed_dict)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
  

<h3>Embeddings</h3>
We have to convert the words in our dataset to vectors of numbers. The traditional approach is to use one-hot encoding method that is usually used for converting categorical values to numerical values. However, One-hot encoded vectors are high-dimensional, sparse and in a big dataset, computationally inefficient. So, we use word2vec approach. It is, in fact, a layer in our LSTM network, where the word IDs will be represented as a dense representation before feeding to the LSTM. 

The embedded vectors also get updated during the training process of the deep neural network.
We create the embeddings for our input data. <b>embedding_vocab</b> is matrix of [10000x200] for all 10000 unique words.

In [18]:
embedding_vocab = tf.get_variable("embedding_vocab", [vocab_size, embeding_vector_size])  #[10000x200]

Lets initialize the <code>embedding_words</code> variable with random values.

In [19]:
session.run(tf.global_variables_initializer())
session.run(embedding_vocab)

array([[ 0.01140717,  0.02023567, -0.01408729, ...,  0.02321866,
        -0.02281097,  0.0064531 ],
       [-0.01239462, -0.01340416,  0.01333335, ..., -0.01969419,
         0.00411844, -0.0067297 ],
       [ 0.0042109 ,  0.01001752, -0.00874475, ..., -0.00572756,
         0.01278892,  0.00889745],
       ...,
       [-0.02324303, -0.01835205, -0.0232351 , ..., -0.01234556,
        -0.00086483,  0.01502647],
       [-0.00759253, -0.01854469,  0.0218081 , ...,  0.00881909,
         0.00319346, -0.01668934],
       [-0.0044563 ,  0.01767663,  0.0168043 , ...,  0.0110224 ,
        -0.00375866, -0.0195694 ]], dtype=float32)

<b>embedding_lookup()</b> finds the embedded values for our batch of 30x20 words. It  goes to each row of <code>input_data</code>, and for each word in the row/sentence, finds the correspond vector in <code>embedding_dic<code>. <br>
It creates a [30x20x200] tensor, so, the first element of <b>inputs</b> (the first sentence), is a matrix of 20x200, which each row of it, is vector representing a word in the sentence.

In [20]:
# Define where to get the data for our embeddings from
inputs = tf.nn.embedding_lookup(embedding_vocab, _input_data)  #shape=(30, 20, 200) 
inputs

<tf.Tensor 'embedding_lookup:0' shape=(60, 20, 200) dtype=float32>

In [21]:
session.run(inputs[0], feed_dict)

array([[ 0.01662669, -0.00851611,  0.01330983, ...,  0.00729107,
        -0.0086226 , -0.02140759],
       [-0.01220327, -0.01397082,  0.00352248, ...,  0.01791978,
         0.0130741 ,  0.01054953],
       [-0.0205128 , -0.00927834,  0.00490582, ...,  0.0176493 ,
         0.01509868,  0.01343441],
       ...,
       [-0.00282781,  0.01959827, -0.01643595, ..., -0.0207481 ,
        -0.01513887,  0.0176754 ],
       [ 0.00578189, -0.0148677 ,  0.00959211, ...,  0.00093682,
         0.00683254,  0.0033809 ],
       [ 0.00332061,  0.01277157,  0.01495042, ..., -0.02184252,
        -0.00828377,  0.00992631]], dtype=float32)

<h3>Constructing Recurrent Neural Networks</h3>
<b>tf.nn.dynamic_rnn()</b> creates a recurrent neural network using <b>stacked_lstm</b>. 

The input should be a Tensor of shape: [batch_size, max_time, embedding_vector_size], in our case it would be (30, 20, 200)

This method, returns a pair (outputs, new_state) where:
<ul>
    <li><b>outputs</b>: is a length T list of outputs (one for each input), or a nested tuple of such elements.</li>
    <li><b>new_state</b>: is the final state.</li>
</ul>

In [22]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

so, lets look at the outputs. The output of the stackedLSTM comes from 200 hidden_layer, and in each time step(=20), one of them get activated. we use the linear activation to map the 200 hidden layer to a [?x10 matrix]

In [23]:
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(60, 20, 128) dtype=float32>

In [24]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[-1.71888416e-04, -1.90442035e-04, -1.21335215e-04, ...,
         2.21405480e-05,  1.08889930e-04,  5.06063749e-04],
       [-4.61957592e-04, -2.51115533e-04,  3.18524049e-04, ...,
         7.01285608e-04,  3.02254222e-04,  7.67858874e-04],
       [-4.44965379e-04, -5.86587936e-04,  5.11239341e-04, ...,
         5.55076229e-04,  7.23287172e-04,  6.18352322e-04],
       ...,
       [ 6.12409320e-04, -3.98175529e-04,  1.65266720e-05, ...,
         9.90393106e-04, -7.42864737e-04, -1.90373365e-04],
       [ 1.09831581e-03, -8.58814514e-04, -5.45308867e-04, ...,
         9.48285277e-04, -6.45909866e-04,  6.32744486e-05],
       [ 8.09730322e-04, -5.29670913e-04, -6.29357179e-04, ...,
         1.45526428e-03, -8.19438777e-04,  9.04873887e-05]], dtype=float32)

we need to flatten the outputs to be able to connect it softmax layer. Lets reshape the output tensor from  [30 x 20 x 200] to [600 x 200].

<b>Notice:</b> Imagine our output is 3-d tensor as following (of course each <code>sen_x_word_y</code> is a an embedded vector by itself): 
<ul>
    <li>sentence 1: [[sen1word1], [sen1word2], [sen1word3], ..., [sen1word20]]</li> 
    <li>sentence 2: [[sen2word1], [sen2word2], [sen2word3], ..., [sen2word20]]</li>   
    <li>sentence 3: [[sen3word1], [sen3word2], [sen3word3], ..., [sen3word20]]</li>  
    <li>...  </li>
    <li>sentence 30: [[sen30word1], [sen30word2], [sen30word3], ..., [sen30word20]]</li>   
</ul>
Now, the flatten would convert this 3-dim tensor to:

[ [sen1word1], [sen1word2], [sen1word3], ..., [sen1word20],[sen2word1], [sen2word2], [sen2word3], ..., [sen2word20], ..., [sen30word20] ]


In [25]:
output = tf.reshape(outputs, [-1, hidden_size_l2])
output

<tf.Tensor 'Reshape:0' shape=(1200, 128) dtype=float32>

<h3>logistic unit</h3>
Now, we create a logistic unit to return the probability of the output word in our vocabulary with 1000 words. 

$$Softmax = [600 \times 200] * [200 \times 1000] + [1 \times 1000] \Longrightarrow [600 \times 1000]$$

In [26]:
softmax_w = tf.get_variable("softmax_w", [hidden_size_l2, vocab_size]) #[200x1000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
logits = tf.matmul(output, softmax_w) + softmax_b
prob = tf.nn.softmax(logits)

Lets look at the probability of observing words for t=0 to t=20:

In [27]:
session.run(tf.global_variables_initializer())
output_words_prob = session.run(prob, feed_dict)
print("shape of the output: ", output_words_prob.shape)
print("The probability of observing words in t=0 to t=20", output_words_prob[0:20])

shape of the output:  (1200, 10000)
The probability of observing words in t=0 to t=20 [[1.01532700e-04 1.00643199e-04 9.89859182e-05 ... 1.01271857e-04
  9.84303551e-05 1.00540215e-04]
 [1.01536200e-04 1.00645731e-04 9.89801410e-05 ... 1.01274352e-04
  9.84306753e-05 1.00541707e-04]
 [1.01547368e-04 1.00649733e-04 9.89855689e-05 ... 1.01275567e-04
  9.84364378e-05 1.00542115e-04]
 ...
 [1.01549078e-04 1.00623103e-04 9.89786859e-05 ... 1.01270205e-04
  9.84297803e-05 1.00543286e-04]
 [1.01543352e-04 1.00628902e-04 9.89790424e-05 ... 1.01269208e-04
  9.84296930e-05 1.00541605e-04]
 [1.01558013e-04 1.00641366e-04 9.89852924e-05 ... 1.01264923e-04
  9.84261424e-05 1.00538244e-04]]


<h3>Prediction</h3>
What is the word correspond to the probability output? Lets use the maximum probability:

In [28]:
np.argmax(output_words_prob[0:20], axis=1)

array([  46,   46, 4687, 2919, 6467, 6467,  352,  352, 3561, 8549, 3868,
       3868, 5674, 7517, 7517, 2139, 9659, 2077, 2077, 7246])

So, what is the ground truth for the first word of first sentence? 

In [29]:
y[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

Also, you can get it from target tensor, if you want to find the embedding vector:

In [30]:
targ = session.run(_targets, feed_dict) 
targ[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

How similar the predicted words are to the target words?

<h4>Objective function</h4>

Now we have to define our objective function, to calculate the similarity of predicted values to ground truth, and then, penalize the model with the error. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}$$

This function is already implemented and available in TensorFlow through <b>sequence_loss_by_example</b>. It calculates the weighted cross-entropy loss for <b>logits</b> and the <b>target</b> sequence.  

The arguments of this function are:  
<ul>
    <li>logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].</li>  
    <li>targets: List of 1D batch-sized int32 Tensors of the same length as logits.</li>   
    <li>weights: List of 1D batch-sized float-Tensors of the same length as logits.</li> 
</ul>

In [31]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

loss is a 1D batch-sized float Tensor [600x1]: The log-perplexity for each sequence. Lets look at the first 10 values of loss:

In [32]:
session.run(loss, feed_dict)[:10]

array([9.204408, 9.224222, 9.202195, 9.224459, 9.195793, 9.21846 ,
       9.19586 , 9.225675, 9.224486, 9.205398], dtype=float32)

Now, we define loss as average of the losses:

In [33]:
cost = tf.reduce_sum(loss) / batch_size
session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)


184.19095

<h4>1. Define Optimizer</h4>

<b>GradientDescentOptimizer</b> constructs a new gradient descent optimizer. Later, we use constructed <b>optimizer</b> to compute gradients for a loss and apply gradients to variables.

In [34]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)


<h4>2. Trainable Variables</h4>

Defining a variable, if you passed <i>trainable=True</i>, the variable constructor automatically adds new variables to the graph collection <b>GraphKeys.TRAINABLE_VARIABLES</b>. Now, using <i>tf.trainable_variables()</i> you can get all variables created with <b>trainable=True</b>.

In [35]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding_vocab:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(456, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(384, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(128, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

Note: we can find the name and scope of all variables:

In [36]:
[v.name for v in tvars]

['embedding_vocab:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0',
 'softmax_w:0',
 'softmax_b:0']

<h4>3. Calculate the gradients based on the loss function</h4>

<h4>Gradient</h4>:
The gradient of a function is the slope of its derivative (line), or in other words, the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by the <b>derivative</b> operation.

First lets recall the gradient function using an toy example:
$$ z = \left(2x^2 + 3xy\right)$$

In [37]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0 * var_x * var_x + 3.0 * var_x * var_y
session.run(tf.global_variables_initializer())
session.run(func_test, {var_x:1.0,var_y:2.0})

8.0

The <b>tf.gradients()</b> function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. <b>tf.gradients(func, xs)</b> constructs symbolic partial derivatives of sum of <b>func</b> w.r.t. <i>x</i> in <b>xs</b>. 

Now, lets look at the derivitive w.r.t. <b>var_x</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 4x + 3y $$


In [38]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad, {var_x:1.0,var_y:2.0})

[10.0]

the derivative w.r.t. <b>var_y</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 3x $$

In [39]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad, {var_x:1.0, var_y:2.0})

[3.0]

Now, we can look at gradients w.r.t all variables:

In [40]:
tf.gradients(cost, tvars)

 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'gradients_2/MatMul_grad/MatMul_1:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'gradients_2/add_grad/Reshape_1:0' shape=(10000,) dtype=float32>]

In [41]:
grad_t_list = tf.gradients(cost, tvars)
#sess.run(grad_t_list,feed_dict)

In [42]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_4:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_5:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_6:0' shape=(10000,) dtype=float32>]

In [43]:
session.run(grads, feed_dict)

[IndexedSlicesValue(values=array([[-2.06744880e-05,  6.64760682e-06,  5.72348654e-06, ...,
         -7.17892681e-06,  6.39652080e-06, -1.54186273e-06],
        [-1.59200627e-05,  2.77802678e-06,  6.61823606e-07, ...,
         -3.91067124e-06,  8.30206409e-06, -1.75804166e-06],
        [-2.84447560e-05,  5.82286202e-06, -3.61220259e-07, ...,
         -2.71707586e-06,  1.12787102e-05, -1.86755574e-06],
        ...,
        [ 1.46565776e-06,  4.07864763e-06, -1.85293447e-05, ...,
         -8.35601156e-07, -5.48072376e-06,  2.27930468e-06],
        [ 5.22765504e-06, -1.01445255e-06, -1.20398945e-05, ...,
          1.34440950e-06, -2.39876982e-07,  1.15070748e-07],
        [ 4.00256113e-06, -1.54685074e-06, -4.91942501e-06, ...,
         -8.29346547e-08, -1.01783417e-06,  3.10083720e-07]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1], dtype=int32), dense_shape=array([10000,   200], dtype=int32)),
 array([[ 1.9072011e-09, -4.0107182e-08, -6.1586021e-09, ...,
      

<h4>4. Apply the optimizer to the variables / gradients tuple.</h4>

In [44]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [45]:
session.run(tf.global_variables_initializer())
session.run(train_op, feed_dict)

<a id="ltsm"></a>
<h2>LSTM</h2>

In [46]:
hidden_size_l1

256

In [47]:
class PTBModel(object):

    def __init__(self, action_type):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l1, forget_bias=0.0)
        lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l2, forget_bias=0.0)
        
        if action_type == "is_training" and keep_prob < 1:
            lstm_cell_l1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l1, output_keep_prob=keep_prob)
            lstm_cell_l2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l2, output_keep_prob=keep_prob)
        
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            embedding = tf.get_variable("embedding", [vocab_size, self.embeding_vector_size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)
        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, self.hidden_size_l2])
        softmax_w = tf.get_variable("softmax_w", [self.hidden_size_l2, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        prob = tf.nn.softmax(logits)
        out_words = tf.argmax(prob, axis=2)
        self._output_words = out_words
        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
            

        # Use the contrib sequence loss and average over the batches
        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            self.targets,
            tf.ones([batch_size, num_steps], dtype=tf.float32),
            average_across_timesteps=False,
            average_across_batch=True)
    
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
#                                                       [tf.ones([batch_size * num_steps])])
        self._cost = tf.reduce_sum(loss)

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if action_type != "is_training":
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data


    
    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state
    
    # Returns the final output words for this model
    @property
    def final_output_words(self):
        return self._output_words
    
    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

With that, the actual structure of our Recurrent Neural Network with Long Short-Term Memory is finished. What remains for us to do is to actually create the methods to run through time -- that is, the <code>run_epoch</code> method to be run at each epoch and a <code>main</code> script which ties all of this together.

What our <code>run_epoch</code> method should do is take our input data and feed it to the relevant operations. This will return at the very least the current result for the cost function.

In [48]:
##########################################################################################################################
# run_one_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_one_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, out_words, _ = session.run([m.cost, m.final_state, m.final_output_words, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Now, we create the <code>main</code> method to tie everything together. The code here reads the data from the directory, using the <code>reader</code> helper module, and then trains and evaluates the model on both a testing and a validating subset of data.

In [49]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [ ]:
# Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale, init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel("is_training")
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel("is_validating")
        mtest = PTBModel("is_testing")

    #Initialize all variables
    tf.global_variables_initializer().run()

    for i in range(max_epoch):
        # Define the decay for this epoch
        lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
        
        # Set the decayed learning rate as the learning rate for this epoch
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        # Run the loop for this epoch in the training model
        train_perplexity = run_one_epoch(session, m, train_data, m.train_op, verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_one_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_one_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
Itr 10 of 774, perplexity: 4431.052 speed: 2008 wps
Itr 87 of 774, perplexity: 1286.832 speed: 2064 wps
Itr 164 of 774, perplexity: 987.177 speed: 2065 wps
Itr 241 of 774, perplexity: 819.230 speed: 2060 wps
Itr 318 of 774, perplexity: 724.194 speed: 2062 wps
Itr 395 of 774, perplexity: 646.241 speed: 2064 wps
Itr 472 of 774, perplexity: 585.237 speed: 2064 wps
Itr 549 of 774, perplexity: 530.994 speed: 2067 wps
Itr 626 of 774, perplexity: 487.721 speed: 2066 wps
Itr 703 of 774, perplexity: 453.391 speed: 2065 wps
Epoch 1 : Train Perplexity: 428.830
Epoch 1 : Valid Perplexity: 233.221
Epoch 2 : Learning rate: 1.000
Itr 10 of 774, perplexity: 267.816 speed: 2013 wps
Itr 87 of 774, perplexity: 235.621 speed: 2041 wps
Itr 164 of 774, perplexity: 226.245 speed: 2053 wps
Itr 241 of 774, perplexity: 216.714 speed: 2056 wps
Itr 318 of 774, perplexity: 213.696 speed: 2058 wps
Itr 395 of 774, perplexity: 207.841 speed: 2060 wps
Itr 472 of 774, perplexity: 203.543 

As you can see, the model's perplexity rating drops very quickly after a few iterations. As was elaborated before, <b>lower Perplexity means that the model is more certain about its prediction</b>. As such, we can be sure that this model is performing well!

-------